In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [7]:
spark = SparkSession.builder.appName("Sumit_Mittal_4_recently_asked_questions").getOrCreate()

## Question 1: There is a parquet file stored in datalake, write a pyspark code to read this file and create a dataframe. Then write a code to remove the duplicate records. Write back the dataframe back to the data lake.

### Solution 1

In [ ]:
source = "path/to/your/parquet/file"

destination = "path/to/your/output/results"

df = spark.read.parquet(source)

df.show()

## remove the duplicates
result_df_no_duplicates =  df.dropDuplicates()

result_df_no_duplicates.show()

## write back the result
result_df_no_duplicates.write.parquet(destination, mode = "overwrite")

## Stopping the SparkSession
spark.stop()

## Question 2: 

#### Input

col1      col2      col3

a          aa        1

a          aa        2

b          bb        5

b          bb        3

b          bb        4

#### Output
col1      col2      col3

a          aa        [1, 2]

b          bb       [5,3, 4]

In [15]:
## Input Data
data = [
    ("a", "aa", 1),
    ("a", "aa", 2),
    ("b", "bb", 5),
    ("b", "bb", 3),
    ("b", "bb", 4),
]

## schema
schema = ["col1", "col2", "col3"]

In [16]:
## Create a dataframe
df = spark.createDataFrame(data, schema)

In [17]:
df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   a|  aa|   1|
|   a|  aa|   2|
|   b|  bb|   5|
|   b|  bb|   3|
|   b|  bb|   4|
+----+----+----+



In [ ]:
def make

In [20]:
df_grouped = df.groupBy(["col1", "col2"]).agg(collect_list("col3").alias("col3"))

In [21]:
df_grouped.show()

+----+----+---------+
|col1|col2|     col3|
+----+----+---------+
|   a|  aa|   [1, 2]|
|   b|  bb|[5, 3, 4]|
+----+----+---------+



In [22]:
# spark.stop()

In [23]:
df_grouped.createOrReplaceTempView("MyTable")

In [24]:
df_table = spark.sql("Select * from MyTable")

In [26]:
df_table.show()

+----+----+---------+
|col1|col2|     col3|
+----+----+---------+
|   a|  aa|   [1, 2]|
|   b|  bb|[5, 3, 4]|
+----+----+---------+



## Question 3

Read the JSON file below:

{
"dept_id": 101, 
"e_id": [10101, 10102, 10103]
}

{
"dept_id": 102, 
"e_id": [10201, 10202]
}

Output:

dept_id     e_id

101         10101

101         10102

101         10103

102         10201

102         10202

In [29]:
json1 = { "dept_id": 101, "e_id": [10101, 10102, 10103] }
json2 = { "dept_id": 102, "e_id": [10201, 10202] }

In [32]:
from pyspark.sql import Row

In [33]:
data = [
    Row(dept_id = json1["dept_id"], e_id = json1["e_id"]),
    Row(dept_id = json2["dept_id"], e_id = json2["e_id"]),
]

In [34]:
df = spark.createDataFrame(data)

In [35]:
df.show()

+-------+--------------------+
|dept_id|                e_id|
+-------+--------------------+
|    101|[10101, 10102, 10...|
|    102|      [10201, 10202]|
+-------+--------------------+



In [38]:
flattened_df = df.withColumn("e2_id", explode("e_id"))

In [39]:
flattened_df.show()

+-------+--------------------+-----+
|dept_id|                e_id|e2_id|
+-------+--------------------+-----+
|    101|[10101, 10102, 10...|10101|
|    101|[10101, 10102, 10...|10102|
|    101|[10101, 10102, 10...|10103|
|    102|      [10201, 10202]|10201|
|    102|      [10201, 10202]|10202|
+-------+--------------------+-----+



## Question 4

data = [( "2023-01-01", "AAPL", 150.00), ("2023-01-02", "AAPL", 155.00),
("2023-01-01", "GOOG", 2500.00), ("2023-01-02", "GOOG", 2550.00),
("2023-01-01", "MSFT", 300.00), ("2023-01-02", "MSFT", 310.00)]

1. Create Dataframe in pyspark

2. Find average stock value on daily basis for each stock

3. Find the max avg stock value for each stock

In [42]:
data = [( "2023-01-01", "AAPL", 150.00), ("2023-01-02", "AAPL", 155.00),
("2023-01-01", "GOOG", 2500.00), ("2023-01-02", "GOOG", 2550.00),
("2023-01-01", "MSFT", 300.00), ("2023-01-02", "MSFT", 310.00)]

In [45]:
df = spark.createDataFrame(data, schema = ["date", "symbol", "value"])

In [46]:
df.show()

+----------+------+------+
|      date|symbol| value|
+----------+------+------+
|2023-01-01|  AAPL| 150.0|
|2023-01-02|  AAPL| 155.0|
|2023-01-01|  GOOG|2500.0|
|2023-01-02|  GOOG|2550.0|
|2023-01-01|  MSFT| 300.0|
|2023-01-02|  MSFT| 310.0|
+----------+------+------+



In [51]:
daily_avg_df = df.groupBy(["date", "symbol"]).agg(avg("value").alias("avg_value"))

In [52]:
daily_avg_df.show()

+----------+------+---------+
|      date|symbol|avg_value|
+----------+------+---------+
|2023-01-01|  AAPL|    150.0|
|2023-01-02|  AAPL|    155.0|
|2023-01-01|  GOOG|   2500.0|
|2023-01-02|  GOOG|   2550.0|
|2023-01-01|  MSFT|    300.0|
|2023-01-02|  MSFT|    310.0|
+----------+------+---------+



In [54]:
max_avg_df = daily_avg_df.groupBy("symbol").agg(max("avg_value").alias("max_avg_value"))

In [55]:
max_avg_df.show()

+------+-------------+
|symbol|max_avg_value|
+------+-------------+
|  AAPL|        155.0|
|  GOOG|       2550.0|
|  MSFT|        310.0|
+------+-------------+



In [57]:
df = df.withColumn("date", to_date(df.date, "yyyy-mm-dd"))

In [58]:
df.show()

+----------+------+------+
|      date|symbol| value|
+----------+------+------+
|2023-01-01|  AAPL| 150.0|
|2023-01-02|  AAPL| 155.0|
|2023-01-01|  GOOG|2500.0|
|2023-01-02|  GOOG|2550.0|
|2023-01-01|  MSFT| 300.0|
|2023-01-02|  MSFT| 310.0|
+----------+------+------+



In [60]:
df = df.withColumn("day", day(df.date))

In [61]:
df.show()

+----------+------+------+---+
|      date|symbol| value|day|
+----------+------+------+---+
|2023-01-01|  AAPL| 150.0|  1|
|2023-01-02|  AAPL| 155.0|  2|
|2023-01-01|  GOOG|2500.0|  1|
|2023-01-02|  GOOG|2550.0|  2|
|2023-01-01|  MSFT| 300.0|  1|
|2023-01-02|  MSFT| 310.0|  2|
+----------+------+------+---+



In [62]:
daily_avg_df = df.groupBy(["day", "symbol"]).agg(avg("value").alias("avg_value"))

In [63]:
daily_avg_df.show()

+---+------+---------+
|day|symbol|avg_value|
+---+------+---------+
|  1|  AAPL|    150.0|
|  2|  AAPL|    155.0|
|  1|  GOOG|   2500.0|
|  2|  GOOG|   2550.0|
|  1|  MSFT|    300.0|
|  2|  MSFT|    310.0|
+---+------+---------+

